In [3]:
def read_file(filepath):
    with open(filepath) as f:
        str_text=f.read()
    return str_text

In [5]:
#read_file('moby_dick_four_chapters.txt')
#read_file('melville-moby_dick.txt')

In [6]:
import spacy

In [7]:
nlp=spacy.load('en',disable=['parser','tagger','ner'])

In [8]:
nlp.max_length=1198623

In [9]:
def separate_punc(doc_text):
    return [token.text.lower()for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [10]:
d=read_file('moby_dick_four_chapters.txt')

In [11]:
tokens=separate_punc(d)

In [13]:
len(tokens)

11338

In [14]:
#25 words ---> network predict #26

In [16]:
train_len=25+1

text_sequences=[]

for i in range(train_len,len(tokens)):
    seq= tokens[i-train_len:i]
    text_sequences.append(seq)

In [21]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [22]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [23]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [29]:
from keras.preprocessing.text import Tokenizer

In [36]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [37]:
sequences=tokenizer.texts_to_sequences(text_sequences)

In [39]:
#sequences

In [41]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [43]:
#tokenizer.word_counts

In [44]:
vocabulary_size=len(tokenizer.word_counts)

In [46]:
vocabulary_size  #unique words

2717

In [47]:
import numpy as np

In [48]:
sequences=np.array(sequences)

In [50]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

# -------------------------------------------

In [52]:
from keras.utils import to_categorical

In [58]:
X=sequences[:,:-1]

In [56]:
Y=sequences[:,-1]

In [57]:
Y=to_categorical(Y,num_classes=vocabulary_size+1)

In [59]:
seq_len=X.shape[1]

In [60]:
X.shape

(11312, 25)

In [61]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [62]:
def create_model(vocabulary_size,seq_len):
    
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(seq_len*2,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocabulary_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    return model

In [63]:
model=create_model(vocabulary_size+1,seq_len)

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2718)              138618    
Total params: 244,518
Trainable params: 244,518
Non-trainable params: 0
_________________________________________________________________


In [64]:
from pickle import dump,load

In [65]:
model.fit(X,Y,batch_size=128,epochs=2,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
11312/11312 [==============================] - 11s 931us/step - loss: 6.9855 - accuracy: 0.0380
Epoch 2/2
11312/11312 [==============================] - 8s 714us/step - loss: 6.3743 - accuracy: 0.0529


In [66]:
model.save('my_moby_dick_model_prac.h5')

In [67]:
dump(tokenizer,open('my_tokenizer','wb'))


# ------------------------------------------------

In [68]:
from keras.preprocessing.sequence import pad_sequences

In [76]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    
    output_text=[]
    
    input_text=seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded=pad_sequences([encoded_text],maxlen=seq_len,truncating='pre')
        
        pred_word_index=model.predict_classes(pad_encoded,verbose=0)[0]
        
        pre_word=tokenizer.index_word[pred_word_index]
        
        input_text += ' '+pre_word
        
        output_text.append(pre_word)
        
    return ' '.join(output_text)

In [77]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [78]:
import random
random.seed(101)
random_pick=random.randint(0,len(text_sequences))

In [79]:
random_seed_text=text_sequences[random_pick]

In [80]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [81]:
seed_text=' '.join(random_seed_text)

In [82]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [83]:
generate_text(model,tokenizer,seq_len,seed_text,num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [84]:
from keras.models import load_model

In [85]:
model=load_model('epochBIG.h5')

In [86]:
tokenizer=load(open('epochBIG','rb'))

In [87]:
generate_text(model,tokenizer,seq_len,seed_text,num_gen_words=25)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing"